In [62]:
import sys
import json

catchment_radius={'bus':'500','subway':'800','rail':'1000'}
params = {'general':catchment_radius}

default = {'training_folder': '../../scenarios/houston', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston', 'params': {'general': {'bus': '500', 'subway': '800', 'rail': '1000'}}}


In [63]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
from typing import Literal

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [64]:
from shapely.geometry import Point, LineString
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

# Folder and parameters

In [65]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [66]:
# Quenedi Parameters are in sended to argv['params']
catchment_radius = argv['params' ]['general']
catchment_radius = {k:float(v) for k,v in catchment_radius.items()}
default_catchment_radius = 500
catchment_radius

{'bus': 500.0, 'subway': 800.0, 'rail': 1000.0}

# Inputs (networks)

In [67]:
links = gpd.read_file(os.path.join(pt_folder, 'links.geojson'), engine=io_engine) 
nodes = gpd.read_file(os.path.join(pt_folder, 'nodes.geojson'), engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

In [68]:
# find meters CRS
centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

/tmp/ipykernel_26427/550055817.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(nodes.centroid.values).centroid.coords][0]


32615

# inputs (zones)

In [69]:
zones_file = os.path.join(input_folder, 'zones.geojson')
zones_file_provided = os.path.isfile(zones_file)
if zones_file_provided :
    zones = gpd.read_file(zones_file, engine=io_engine)
    if 'index' in zones.columns:
        zones = zones.set_index('index')
    else:
        zones.index.name='index'
    assert 'density' in zones.columns, 'need density column. in km2'
    assert zones.crs == 4326, 'population.geojson CRS must be EPSG:4326'
print('zones?',zones_file_provided)

zones? True


In [70]:
if zones_file_provided : 
    zones['area (km2)'] = zones.to_crs(crs).area*1e-6
    zones['area (km2)'].sum()
    zones['population'] = zones['density']*zones['area (km2)']
    print(zones['population'].sum())

7212861.251305611


# Outputs

In [6]:
#links.to_file(output_folder + 'links.geojson',driver='GeoJSON', engine=io_engine)